In [45]:
import pandas as pd
adrian = pd.read_csv('label_text_Adrian.csv', header = None)
adrian

,0
0,Boneless Lamb Product of Australia L Leg-Chump...
1,Keep Frozen Product of Australia Beef Burger B...
2,Premium Cooked Frozen Octopus Tentacles Sous V...
3,Frozen Ocean Trout Fillet Skin On Individually...
4,Gourmet Beef Burger Patties Part Cooked 120g T...
5,Tassal Tasmanian Salmon Product of Australia T...
6,SL SFRC Date 4/07/24 Packed Snapper FLTS Red C...
7,Frozen Thigh Fillet N/B-Diced 94234244564 Prod...
8,(30PCS/BX) Frozen Refrigeration: Keep Frozen(S...
9,SEFSSFFF Packed Date 25/06/24 Snapper FLTS Sad...


In [497]:
import pandas as pd
warra = pd.read_csv('label_text_Warrawunda.csv',header=None)
warra

,0
0,West Coast Meat Solutions PLU 10316T BBQ Sausa...
1,Beef Mince 80cl Frozen (KG) Packed On Best Bef...
2,26 Boneless Beef Mince 90VL 5mm 3 x 5kg Pieces...
3,Chilled Keep Refrigerated - If Frozen Keep Frozen
4,"Avon Valley Beef 503 Northam Pithara Road, Pit..."
5,Product of Australia Boneless Beef 01199036538...
6,Keep Frozen Product of Australia 0030000000001...
7,West Coast Meat Solutions PLU 10718T VAC Piece...
8,47 Boneless Beef Yearling Diced Large 3x5kg Fr...
9,Chilled Keep Refrigerated - If Frozen Keep Frozen


In [46]:
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch
modelname = 'bert-large-uncased-whole-word-masking-finetuned-squad'
tokenizer = AutoTokenizer.from_pretrained(modelname)
model = AutoModelForQuestionAnswering.from_pretrained(modelname)

C:\Users\Loc Sensei\anaconda\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [537]:
from datetime import datetime, timedelta
import re
questions = [
    "What is the name of the food ?",
    "What is the total weight ?",
    "What is the expiry date ?"]

# Function to compute the best before date from packed date
def compute_best_before_date(packed_date, months):
    try:
        packed_date_obj = datetime.strptime(packed_date, "%d/%m/%y")  # Handle two-digit year
        best_before_date_obj = packed_date_obj + timedelta(days=months*30)  # Approximate month length as 30 days
        return best_before_date_obj.strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error computing best before date: {e}")
        return None

# Function to parse dates in DDMMYY format
def parse_ddmmyy(date_str):
    try:
        return datetime.strptime(date_str, "%d%m%y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None

# Function to parse best before date in YYMMDD format
def parse_yymmdd(date_str):
    try:
        return datetime.strptime(date_str, "%d%m%y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None

# Function to parse dates in DD.MMM.YYYY format
def parse_ddmmmyyyy(date_str):
    try:
        return datetime.strptime(date_str, "%d.%b.%Y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    
# Function to parse dates in DD.MM.YYYY format
def parse_ddmmyyyy(date_str):
    try:
        return datetime.strptime(date_str, "%d.%m.%Y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    
# Function to parse dates in DD MMM YY format
def parse_ddmmmyy(date_str):
    try:
        return datetime.strptime(date_str, "%d %b %y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    

# Function to parse dates in DD-MMM-YYYY format
def parse_ddmmmyyyy_hyphen(date_str):
    try:
        return datetime.strptime(date_str, "%d-%b-%Y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None
    
# Function to parse dates in DD-MM-YY format
def parse_ddmmyy_hyphen(date_str):
    try:
        return datetime.strptime(date_str, "%d-%m-%y").strftime("%d/%m/%Y")
    except ValueError as e:
        print(f"Error parsing date {date_str}: {e}")
        return None

In [538]:
# Function to get answer for a single question
def get_answer(question, context):
    # Prepare the inputs
    inputs = tokenizer.encode_plus(question, context, return_tensors='pt')
    input_ids = inputs["input_ids"].tolist()[0]

    # Get model outputs
    outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits

    # Get the most likely beginning and end of answer span
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1

    # Decode the answer
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

    
    if "total weight" in question.lower():
        # First, look for multiple quantities and weights
        weights = re.findall(r'(\d+)\s*x\s*(\d+\.?\d*)\s*kg', context, re.IGNORECASE)
        total_weight = 0
        for quantity, weight in weights:
            total_weight += int(quantity) * float(weight)
        if total_weight > 0:
            return f"{total_weight:.2f} kg"
        else:
            # If no multiple quantities are found, look for single weight entries
            match = re.search(r'(Net Weight|Net Wt|Net Ungt):?\s*(\d+\.?\d*)', context, re.IGNORECASE)
            if match:
                return f"{match.group(2)} kg"
            else:
                match = re.search(r'(\d+\.?\d*)\s*kg', context, re.IGNORECASE)
                if match:
                    return f"{match.group(1)} kg"
                else:
                    return "Total weight information not found."
    
        
    elif any(term in question.lower() for term in ["expiry date", "use by date", "best before date", "expiry"]):
        # Check for standard date format
        date_match_standard = re.search(r'(Use By|Best Before|Expiry Date):?\s*(\d{2}/\d{2}/\d{4})', context, re.IGNORECASE)
        if date_match_standard:
            return date_match_standard.group(2)  # Return the extracted date

        # Check for long date format
        date_match_long = re.search(r'(Use By|Best Before|Expiry Date):?\s*(\d{1,2} \w+ \d{4})', context, re.IGNORECASE)
        if date_match_long:
            return date_match_long.group(2)  # Return the extracted long format date

        # Check if production date has "date" word before and best before random int months
        packed_date_match = re.search(r'Date (\d{1,2}/\d{1,2}/\d{2,4})', context, re.IGNORECASE)
        best_before_match = re.search(r'Best Before (\d+) Months from Packed Date', context, re.IGNORECASE)
        if packed_date_match and best_before_match:
            packed_date = packed_date_match.group(1)
            months = int(best_before_match.group(1))
            best_before_date = compute_best_before_date(packed_date, months)
            return best_before_date
        
        # Check if production date has no "date" word before and best before random int months
        date_match_unlabeled = re.search(r'\b(\d{1,2}/\d{1,2}/\d{2,4})\b', context)
        best_before_match = re.search(r'Best Before (\d+) Months from Packed Date', context, re.IGNORECASE)
        if date_match_unlabeled and best_before_match:
            packed_date = date_match_unlabeled.group(1)
            months = int(best_before_match.group(1))
            best_before_date = compute_best_before_date(packed_date, months)
            return best_before_date
        
        # Check for DDMMYY format
        date_match_ddmmyy = re.search(r'Best Before:\s*(\d{6})', context, re.IGNORECASE)
        if date_match_ddmmyy:
            best_before_date = parse_ddmmyy(date_match_ddmmyy.group(1))
            return best_before_date
        
        # Check for YYMMDD format
        date_match_yymmdd = re.search(r'Best Before:\s*(\d{6})', context, re.IGNORECASE)
        if date_match_yymmdd:
            best_before_date = parse_yymmdd(date_match_yymmdd.group(1))
            return best_before_date
        
        # Check for DD.MMM.YYYY format
        date_match_ddmmmyyyy = re.search(r'Best Before\s*(\d{2}\.\w{3}\.\d{4})', context, re.IGNORECASE)
        if date_match_ddmmmyyyy:
            best_before_date = parse_ddmmmyyyy(date_match_ddmmmyyyy.group(1))
            return best_before_date
        
        # Check for DD.MM.YYYY format
        date_match_ddmmyyyy = re.search(r'(Use By|Best Before|Expiry Date):?\s*(\d{2}\.\d{2}\.\d{4})', context, re.IGNORECASE)
        if date_match_ddmmyyyy:
            best_before_date = parse_ddmmyyyy(date_match_ddmmyyyy.group(2))
            return best_before_date
        
        # Check for DD MMM YY format
        date_match_ddmmmyy = re.search(r'(Use By|Best Before|Expiry Date):?\s*(\d{2} \w{3} \d{2})', context, re.IGNORECASE)
        if date_match_ddmmmyy:
            best_before_date = parse_ddmmmyy(date_match_ddmmmyy.group(2))
            return best_before_date
        
        # Check for DD-MMM-YYYY format
        date_match_ddmmmyyyy_hyphen = re.search(r'(Use By|Best Before|Expiry Date):?\s*(\d{2}-\w{3}-\d{4})', context, re.IGNORECASE)
        if date_match_ddmmmyyyy_hyphen:
            best_before_date = parse_ddmmmyyyy_hyphen(date_match_ddmmmyyyy_hyphen.group(2))
            return best_before_date
        
        # Check for DD-MM-YY format
        date_match_ddmmyy_hyphen = re.search(r'(Use By|Best Before|Expiry Date|Expiry):?\s*(\d{2}-\d{2}-\d{2})', context, re.IGNORECASE)
        if date_match_ddmmyy_hyphen:
            best_before_date = parse_ddmmyy_hyphen(date_match_ddmmyy_hyphen.group(2))
            return best_before_date
        
        # Check for unlabeled date formats
        date_match_unlabeled = re.search(r'\b(\d{2}/\d{2}/\d{4})\b', context)
        if date_match_unlabeled:
            return date_match_unlabeled.group(1)
        
        return "There's no expiry date."
    
    elif "food" in question.lower() or "product" in question.lower():
        # Extract the product name or food description
        match = re.search(r'Packed Date \d{1,2}/\d{1,2}/\d{2,4} (.+?) (?:\d+\s*KG|Keep Frozen|Product of Australia)', context, re.IGNORECASE)
        if match:
            product_description = match.group(1).strip()
            # Remove any trailing "Product of Australia" if present
            product_description = re.sub(r'\bProduct of Australia\b', '', product_description, flags=re.IGNORECASE).strip()
            return product_description

    return answer.strip()

In [540]:
context = adrian[0][2]

for question in questions:
    answer = get_answer(question, context)
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")

Question: What is the name of the food ?
Answer: frozen octopus tentacles sous vide

Question: What is the total weight ?
Answer: 10.00 kg

Question: What is the expiry date ?
Answer: 24/06/2026



In [533]:
warra[0][2]

'26 Boneless Beef Mince 90VL 5mm 3 x 5kg Pieces 3 Pallet No. Packed on Best Before 058994 24/01/24 23/01/25 Net Weight 15.30 kg Packed for P-590 Frozen of Australia'

In [529]:

for question in questions:
    for context in warra[0]:
        answer = get_answer(question, context)
        print(f"Question: {question}")
        print(f"Answer: {answer}\n")

Question: What is the expiry date ?
Answer: 28/03/2025

Question: What is the expiry date ?
Answer: There's no expiry date.

Question: What is the expiry date ?
Answer: There's no expiry date.

Question: What is the expiry date ?
Answer: There's no expiry date.

Question: What is the expiry date ?
Answer: 27 Jan 2025

Question: What is the expiry date ?
Answer: 25/07/2024

Question: What is the expiry date ?
Answer: 10 Jul 2025

Question: What is the expiry date ?
Answer: 18/12/2023

Question: What is the expiry date ?
Answer: There's no expiry date.

Question: What is the expiry date ?
Answer: There's no expiry date.

Question: What is the expiry date ?
Answer: 10/08/2024

Question: What is the expiry date ?
Answer: 19/06/2024

Question: What is the expiry date ?
Answer: 16 Jun 2023

Question: What is the expiry date ?
Answer: 22/01/2024

Question: What is the expiry date ?
Answer: 03/05/2025

Question: What is the expiry date ?
Answer: 12/10/2023

Question: What is the expiry date ?
